# Likelihood-Free Approach to scRNA-seq Parameter Estimation

Inspired by likelihood free approaches used in population genetics.

### Imports

In [11]:
import torch
import numpy as np
import scipy.stats as stats
from sklearn.datasets import make_spd_matrix

### Create a super simple 2 layer MLP

In [2]:
class MLP(torch.nn.Module):
    
    def __init__(self, D_in, H, D_out):
        
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

### Train

In [204]:
def generate_batch(batch_size=1000):
    
    data_list, mu_list, cov_list = [], [], []
    
    for i in range(batch_size):
        
        cov = make_spd_matrix(2).astype(np.float32)
        
        mu = np.array([np.random.normal(0, 5), np.random.normal(0, 5)]).astype(np.float32)
        
        data = np.random.multivariate_normal(mu, cov, 50).reshape(-1).astype(np.float32)
        
        cov_list.append(cov.reshape(-1).copy())
        mu_list.append(mu.copy())
        data_list.append(data.copy())
    return (
        torch.from_numpy(np.vstack(data_list)), 
        torch.from_numpy(np.vstack(mu_list)), 
        torch.from_numpy(np.vstack(cov_list)[:, 1].reshape(-1, 1)))

In [241]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
D_in, H, D_out = 100, 100, 2
num_iter = 500

# Construct our model by instantiating the class defined above.
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

# x = torch.randn(100, D_in)
# y = torch.randn(100, D_out)

for t in range(num_iter):
    
    if t % 20 == 0:
        print(t, loss.item())
        
    
    x, y, cov = generate_batch()
    
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 0.3764576315879822


/anaconda3/envs/scvi/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: covariance is not positive-semidefinite.


20 12.691710472106934
40 5.274765968322754
60 2.2114176750183105
80 1.0639927387237549
100 0.5689888596534729
120 0.3806800842285156
140 0.2568187713623047
160 0.19561246037483215
180 0.20607924461364746


KeyboardInterrupt: 

In [244]:
test_mu = torch.tensor([3.4, 2])
test_cov = torch.tensor([[2.05,1], [1, 0.4]])

In [282]:
data = torch.from_numpy(np.random.multivariate_normal(test_mu, test_cov, 50).reshape(-1).astype(np.float32))
model(data)

/anaconda3/envs/scvi/lib/python3.6/site-packages/ipykernel/__main__.py:1: RuntimeWarning: covariance is not positive-semidefinite.
  if __name__ == '__main__':


tensor([3.3202, 2.2393], grad_fn=<ThAddBackward>)

In [208]:
model(data)

tensor([0.2989], grad_fn=<ThAddBackward>)